[Node 64: Programme der Python Standard Library](http://www-static.etp.physik.uni-muenchen.de/kurs/Computing/python2/node64.html)

Navigation:

**Next:** [Verteilte Programme mit XML-RPC](node65.ipynb) **Up:** [Netzwerk Programmierung](node62.ipynb) **Previous:** [Überblick Netzwerkprogrammierung](node63.ipynb)

##  Module der Python-Standardbibliothek

Klassische Anwendungen wie FTP, Versand und Empfangen von E-Mails oder Telnet können synchron abgearbeitet werden, d.h. eine Anforderung wird an einen Server gesendet, auf dessen Antwort gewartet und dann weitergearbeitet. Für zahlreiche Netzwerkdienste existieren Module in der Python-Standardbibliothek wie z.B.  ``ftplib, poplib, imaplib, smtplib, nntplib, gopherlib, telnetlib`` .  

### Server mit SocketServer

Wir gehen hier nicht auf die [Berkeley-Socket-API](https://en.wikipedia.org/wiki/Berkeley_sockets) ein, sondern beschränken uns auf die Klassen des  ``SocketServer``-Moduls, die die Komplexität der *low-level* Socket-API verbergen. 

Bei einem ``SocketServer``  geht man wie folgt vor:  
* Leite Verarbeitungsklasse von  [``BaseRequestHandler``](https://docs.python.org/3/library/socketserver.html#socketserver.BaseRequestHandler)  ab und  überschreibe die  ``handle``  Methode, um eingehende Anfragen ([``socket``](https://docs.python.org/3/library/socket.html#socket-objects)) zu bearbeiten
* Benutze Serverklasse  [``TCPServer``](https://docs.python.org/3/library/socketserver.html#socketserver.TCPServer), [``UDPServer``](https://docs.python.org/3/library/socketserver.html#socketserver.UDPServer)  und instanziiere diese mit Server-Adresse und Verarbeitungsklasse
* Starte Ereignisschleife des Servers.

Ein Beispiel für einen einfachen Server auf TCP-Basis:

In [ ]:
from socketserver import TCPServer, BaseRequestHandler

class HelloHandler(BaseRequestHandler):
    def handle(self):
        print ("Serving client:", self.client_address)
        self.request.sendall(b'Hello Client! I am a HelloHandler\r\n')
        
PORT = 7070
TCPServer.allow_reuse_address = True
srv = TCPServer(("", PORT), HelloHandler)
try:
    srv.serve_forever()
    print ("Serving forever at port", PORT)
except:
    print ("Closing the server.")
    srv.server_close()


Man führt dieses Programm auf dem Server aus und kontaktiert das Server-Programm von einem Client auf der Shell-Kommando-Zeile mit ``nc (netcat)``:

```bash
$ nc localhost 7070
Hello Client! I am a HelloHandler
```

Auf dem Server erhält man bei dieser Verbindung eine Meldung:
<pre>
Serving client: ('127.0.0.1', 48908)
</pre>
(siehe oben...)

---
Wir schreiben uns im folgenden Beispiel einen Echo-Server, der die Anfragen von Klienten unverändert zurückschickt. (Praktisch typischerweise zum Testen.)
Dazu verwenden wir die [`StreamRequestHandler`](https://docs.python.org/3/library/socketserver.html#socketserver.StreamRequestHandler)-Klasse als Basis für unseren Handler, die wiederum von der oben verwendeten Klasse `BaseRequestHandler` ableitet, aber noch zwei Eigenschaften bereitstellt, `self.rfile` und `self.wfile`, über die wir Daten von den Klienten empfangen und an sie senden können. Dabei wollen wir unserem Echo ein `"S:"` voranstellen, um es als Serverantwort kenntlich zu machen. 

#### Exkurs: Textkodierung

Da `line` (in Python 3) vom Type `bytes` ist (eine Abfolge von Bytes), müssen wir dem String `"S:"` (der einen Text repräsentiert und implizit eine bestimmte Kodierung hat, z.B. [UTF-8](https://de.wikipedia.org/wiki/UTF-8)) als Präfix ein `b` voranstellen, um ihn in den Typ `bytes` umzuwandeln:

In [ ]:
type("S:")

In [ ]:
type(b"S:")

Mit expliziter Enkodierung:
([`encode()`](https://docs.python.org/3/library/stdtypes.html#str.encode): `string` $\rightarrow$ `bytes`,
 [`decode()`](https://docs.python.org/3/library/stdtypes.html#bytes.decode): `bytes` $\rightarrow$ `string`)

In [ ]:
type("S:".encode("UTF-8"))

Eine unpassende Textkodierung führt zu dem typischen Bild von kaputten Umlauten:

In [ ]:
"Süßes Tiramisù".encode("UTF-8").decode("iso-8859-15")

In [ ]:
"Süßes Tiramisù".encode("iso-8859-15").decode("utf-8", errors="replace")

In [ ]:
"Süßes Tiramisù".encode("UTF-8").decode("UTF-8")

In [ ]:
"Süßes Tiramisù".encode("iso-8859-1").decode("iso-8859-1")

Zurück zu unserem Echo-Server:

In [ ]:
from socketserver import TCPServer, StreamRequestHandler

class EchoHandler(StreamRequestHandler):
    def handle(self):
        print("Serving client:", self.client_address)
        for line in (self.rfile):
            reply = b"S:" + line
            self.wfile.write(reply)
            
PORT = 7071
TCPServer.allow_reuse_address = True
srv = TCPServer(("", PORT), EchoHandler)
try:
    srv.serve_forever()
    print ("Serving forever at port", PORT)
except:
    print ("Closing the server.")
    srv.server_close()

Startet man obigen Server, so erhält man auf dem Klienten folgende Ein- und Ausgabe:
<pre>
$ nc localhost 7071
Hallo
S:Hallo
Echo !
S:Echo !
</pre>

Der Nachteil des gerade vorgestellten Echo-Servers liegt in der Tatsache, dass er nur einen Klienten zur gleichen Zeit bedienen kann, da der Server nur mit einem Thread läuft. 
Als Lösung bietet sich ein asynchroner Server (wie `Twisted`) an, oder man führt die  ``handle``-Methode in einem eigenen Prozess oder Thread aus. 

Im folgenden Beispiel wird  ``ThreadingTCPServer``  anstatt  ``TCPServer``  verwendet:

In [ ]:
from socketserver import ThreadingTCPServer, StreamRequestHandler

class EchoHandler(StreamRequestHandler):
    def handle(self):
        print("Serving client:", self.client_address)
        for line in (self.rfile):
            reply = b"S:" + line
            self.wfile.write(reply)
            
PORT = 7072
ThreadingTCPServer.allow_reuse_address = True
srv = ThreadingTCPServer(("", PORT), EchoHandler)
try:
    srv.serve_forever()
    print ("Serving forever at port", PORT)
except:
    print ("Closing the server.")
    srv.server_close()

Randbemerkung: Der ``ThreadingTCPServer`` ist ein Beispiel für die früher erwähnten "mix-ins". Seine Definition ist folgende Zeile:
```python
class ThreadingTCPServer(ThreadingMixIn, TCPServer): pass

```

### E-Mails senden mit `smtplib.SMTP`

Um E-Mails synchron zu senden, kann man die Klasse  ``SMTP``  aus dem  ``smtplib``-Modul verwenden. In folgendem Programm wird demonstriert, wie man eine E-Mail mit dem  ``email.message``-Modul zunächst komponiert und sich dann am Mailserver der LMU-Physik authentifiziert und die E-Mail über das SMTP-Protokoll (Simple Mail Transfer Protocol) an seine eigene Adresse versendet: 
[``test_send_mail.py``](source/test_send_mail.py) 

Das Passwort wird über die Funktion ``getpass`` zur Laufzeit des Programms eingelesen.

Das SMTP stammt ursprünglich aus dem Jahr 1982 und ist sehr simpel, aber immer noch die Grundlage von allem Mailverkehr im Internet. 
Es ist so geschrieben, dass über einen kleinen Satz von einfachen Kommandos ein Mensch  interaktiv in einer Serverbindung eine Mail verfassen und verschicken kann. 
Es sieht außerdem keine Authentifizierung vor. Jeder Server kann im Prinzip an jeden anderen Server Mails schicken, ohne beweisen zu müssen, dass er dazu berechtigt ist, oder dass die Mails, die er weiterleitet, tatsächlich von den behaupteten Absendern stammen. Diese Eigenheit im Design des Protokolls ist zwar im Geiste eines freien und offenen Internets, führte aber auch zu dem riesigen Problem mit Spam-Mails, das wir heute haben.

### E-Mails lesen mit `imaplib.IMAP4`

Über die  ``IMAP4_SSL``-Funktion (IMAP = Internet Message Access Protocol) des  ``imaplib``-Moduls können E-Mails von einem Server gelesen werden: [``imaplib_imap4client.py``](source/imaplib_imap4client.py)